# Notebook Olist – Previsão de Tempo de Entrega

In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## 1. Upload dos arquivos no Colab

In [2]:

from google.colab import files
uploaded = files.upload()


Saving olist_customers_dataset.csv to olist_customers_dataset.csv
Saving olist_geolocation_dataset.csv to olist_geolocation_dataset.csv
Saving olist_order_items_dataset.csv to olist_order_items_dataset.csv
Saving olist_orders_dataset.csv to olist_orders_dataset.csv
Saving olist_products_dataset.csv to olist_products_dataset.csv
Saving olist_sellers_dataset.csv to olist_sellers_dataset.csv


## 2. Carregamento e preparação do dataset Olist

In [3]:
## 2. Carregamento e preparação do dataset Olist (Versão Final com Sazonalidade)

import pandas as pd
import numpy as np

# 1. Carregar todos os datasets necessários
orders = pd.read_csv("olist_orders_dataset.csv")
items = pd.read_csv("olist_order_items_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")


# 2. Preparação da Tabela Principal
# Juntar orders, items, customers, sellers e products
df = items.merge(orders, on="order_id", how="inner")
df = df.merge(customers, on="customer_id", how="inner")
df = df.merge(sellers, on="seller_id", how="left")
df = df.merge(products[['product_id', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']],
              on="product_id", how="left")


# 3. Cálculo da Variável Alvo e Limpeza
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

# Remover linhas sem data de entrega
df = df.dropna(subset=['order_delivered_customer_date'])

# Calcular o tempo de entrega em minutos
df['Tempo_Entrega_Minutos'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 60

# Filtrar outliers: focar em entregas entre 1 e 59 dias (1440 min a 84960 min)
df = df[(df['Tempo_Entrega_Minutos'] >= 1440) & (df['Tempo_Entrega_Minutos'] <= 84960)]

# 4. Preparação das Features de Localização (Cálculo de Distância)
# Reduzir o dataset de geolocalização para uma coordenada por CEP (média)
geo_agg = geolocation.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

# Renomear colunas para cliente e vendedor
geo_cliente = geo_agg.rename(columns={'geolocation_zip_code_prefix': 'customer_zip_code_prefix',
                                      'geolocation_lat': 'customer_lat',
                                      'geolocation_lng': 'customer_lng'})

geo_vendedor = geo_agg.rename(columns={'geolocation_zip_code_prefix': 'seller_zip_code_prefix',
                                       'geolocation_lat': 'seller_lat',
                                       'geolocation_lng': 'seller_lng'})

# Merge com o DF principal
df = df.merge(geo_cliente, on='customer_zip_code_prefix', how='left')
df = df.merge(geo_vendedor, on='seller_zip_code_prefix', how='left')

# Remover linhas que não puderam ser geolocalizadas
df = df.dropna(subset=['customer_lat', 'seller_lat'])


# Função para calcular a distância de Haversine em KM
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em km
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Aplicar a função para calcular a distância
df['Distancia_KM'] = haversine(
    df['customer_lat'], df['customer_lng'],
    df['seller_lat'], df['seller_lng']
)

# 5. Novas Features de Produto e Temporais
# Tempo Prometido em Minutos
df['Tempo_Prometido_Minutos'] = (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 60

# Feature de Volume (produto das dimensões)
df['Volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

# Dia da Semana (0=Segunda, 6=Domingo)
df['Dia_Semana'] = df['order_purchase_timestamp'].dt.dayofweek

# Hora da Compra
df['Hora_Compra'] = df['order_purchase_timestamp'].dt.hour

# NOVAS FEATURES DE SAZONALIDADE
df['Mes_Compra'] = df['order_purchase_timestamp'].dt.month
df['Dia_do_Ano'] = df['order_purchase_timestamp'].dt.dayofyear


# 6. Seleção Final das Features para o Modelo
df_final = df[[
    'Distancia_KM',
    'freight_value',
    'Dia_Semana',
    'Hora_Compra',
    'product_weight_g',
    'Volume_cm3',
    'Tempo_Prometido_Minutos',
    'Mes_Compra',         # NOVO
    'Dia_do_Ano',         # NOVO
    'Tempo_Entrega_Minutos' # Variável Alvo
]]

# Lidar com valores NaN
df_final = df_final.dropna()

print("DataFrame final após processamento com 9 features:")
print(df_final.head())

DataFrame final após processamento com 9 features:
   Distancia_KM  freight_value  Dia_Semana  Hora_Compra  product_weight_g  \
0    301.504681          13.29           2            8             650.0   
1    585.563937          19.93           2           10           30000.0   
2    312.343511          17.87           6           14            3050.0   
3    293.168420          12.79           2           10             200.0   
4    646.163463          18.14           5           13            3750.0   

   Volume_cm3  Tempo_Prometido_Minutos  Mes_Compra  Dia_do_Ano  \
0      3528.0             22500.966667           9         256   
1     60000.0             26706.900000           4         116   
2     14157.0             30806.483333           1          14   
3      2400.0             16679.416667           8         220   
4     42000.0             58202.150000           2          35   

   Tempo_Entrega_Minutos  
0           10964.766667  
1           23351.300000  
2       

## 3. Treinamento do modelo

In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Definição de X (9 Features) e y
X = df_final[['Distancia_KM', 'freight_value', 'Dia_Semana', 'Hora_Compra', 'product_weight_g', 'Volume_cm3', 'Tempo_Prometido_Minutos', 'Mes_Compra', 'Dia_do_Ano']]
y = df_final['Tempo_Entrega_Minutos']

# Split e Scaling
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Arquitetura da Rede Neural (Mais Larga com Regularização Dropout)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# COMPILAÇÃO: Loss 'mae'
model.compile(optimizer='adam',loss='mae')

# Treinamento
history = model.fit(X_train_scaled,y_train,epochs=10,validation_split=0.2)

# Avaliação
pred = model.predict(X_test_scaled)
mae_minutos = mean_absolute_error(y_test,pred)

# --- CONVERSÃO AUTOMÁTICA ADICIONADA AQUI ---
mae_dias = mae_minutos / (60 * 24)

print("--- Resultado Final ---")
print(f"MAE em Minutos: {mae_minutos:.4f}")
print(f"MAE em Dias: {mae_dias:.4f}")
print(f"O modelo erra em média {mae_dias:.2f} dias.")

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2186/2186 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 10608.4746 - val_loss: 6892.2549
Epoch 2/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 7086.6533 - val_loss: 6802.2046
Epoch 3/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6919.1250 - val_loss: 6711.3965
Epoch 4/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 6857.9775 - val_loss: 6645.3311
Epoch 5/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6866.3896 - val_loss: 6622.0166
Epoch 6/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6799.4824 - val_loss: 6605.0342
Epoch 7/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6794.0103 - val_loss: 6601.9229
Epoch 8/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6765.3389 - val_loss: 6599.1860
Epoch 9/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6780.0762 - val_loss: 6595.7520
Epoch 10/10
2186/2186 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6713.0625 - val_loss: 6599.0972
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
--- Resultado Final ---
MAE 

## 4. Avaliação